In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from datetime import datetime

In [2]:
path = "C:/Users/Francis John Picaso/Repositories/Teo/toTest"

fullPaths = [folder for folder in listdir(path)]
fullPaths

['220522_110503_AE211776-OPER7220_Slot02_',
 '220522_111858_AE211776-OPER7220_Slot03_',
 '220522_113232_AE211776-OPER7220_Slot04_',
 '220522_114606_AE211776-OPER7220_Slot05_',
 '220522_115939_AE211776-OPER7220_Slot06_',
 '220523_034430_E222056-7226_Slot02_',
 '220523_040517_E222056-7226_Slot03_',
 '220523_042320_E222056-7226_Slot04_',
 '220523_044124_E222056-7226_Slot05_',
 '220523_045932_E222056-7226_Slot06_',
 '220523_121500_AL201894-B_Slot14_']

In [3]:
logInfo = pd.DataFrame()
logInfo["SlotFolder"] = [path + "/" + f for f in fullPaths]
lots = [f.split("_")[2] for f in fullPaths]
logInfo["Lot"] = lots
logInfo["Date"] = ["20" + f.split("_")[0] for f in fullPaths]
logInfo["Time"] = [f.split("_")[1] for f in fullPaths]
logInfo["DateTime"] = logInfo["Date"] + " " + logInfo["Time"]
logInfo['Date'] = logInfo['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
logInfo['DateTime'] = logInfo['DateTime'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d %H%M%S'))
logInfo['DateTimeStr'] = logInfo['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
logInfo["Date"] = logInfo['DateTime'].dt.date
logInfo["Time"] = logInfo['DateTime'].dt.time


logInfoSorted = logInfo.sort_values(['Lot', 'DateTime'],
                  ascending=[True, True])
logInfoSorted['rownumber'] = logInfo.groupby(['Lot']).cumcount()
logInfoSorted['countnumber'] = logInfo.groupby(['Lot'])['DateTime'].transform('count')

In [4]:
lots = logInfoSorted["Lot"].unique()
lots

array(['AE211776-OPER7220', 'AL201894-B', 'E222056-7226'], dtype=object)

In [5]:
columns = ["LotID","CarrierID","TimeStamp","Message","WaferID","SlotNumber","Date","Time"]

lotdf = pd.DataFrame(columns = columns)

for lot in lots:
    slotdata = logInfoSorted[logInfoSorted["Lot"] == lot]
    timestamp = slotdata[slotdata["rownumber"] == 0].DateTime.values[0]
    date = slotdata[slotdata["rownumber"] == 0].Date.values[0]
    time = slotdata[slotdata["rownumber"] == 0].Time.values[0]

    
    lotStart = {
        "LotID": [lot],
        "CarrierID": [""],
        "TimeStamp": [timestamp],
        "Message": ["Lot Start"],
        "WaferID": [""],
        "SlotNumber": [""],
        "Date": [date],
        "Time": [time]
    }
    df_lotstart = pd.DataFrame.from_dict(lotStart)
    lotdf = pd.concat([lotdf,df_lotstart])

    
    for index, row in slotdata.iterrows():
        csvPath = row["SlotFolder"] + "/Info/"
        files = [f for f in listdir(csvPath) if isfile(join(csvPath, f))]
        for file in files:
            slotNum = int(file.split("_")[0].replace("Slot",""))
            WaferID = "@" + str(slotNum)
           
            file = csvPath + file
            columnPlaceholder = ['col' + str(i) for i in range(10)]
            df = pd.read_csv(file,names=columnPlaceholder)
            carrier_id = df[df["col0"] == "Carrier ID"]["col1"].values[0]
            
            #now get the date
            #look for Tip Slot No
            index = df[df["col0"] == "Tip Slot No"].index[0]
            steps_header = index + 1
            new_header = df.iloc[steps_header] #grab the first row for the header
            steps = df[steps_header + 1:] #take the data less the header row
            steps.columns = new_header #set the header row as the df header
            end_date_time = steps.iloc[-1:]["Date"].values[0]
            end_date_time =  datetime.strptime(end_date_time, '%Y/%m/%d %H:%M:%S')
            #print("Lot ID: ", lot,"Carrier ID:", carrier_id,"End Date:" , end_date )
            
            lotFileInfo = {
                "LotID": [lot],
                "CarrierID": [carrier_id],
                "TimeStamp": [end_date_time],
                "Message": [""],
                "WaferID": [WaferID],
                "SlotNumber": [slotNum],
                "Date": [end_date_time.date()],
                "Time": [end_date_time.time()]
            }
            df_lotfileInfo = pd.DataFrame.from_dict(lotFileInfo)
            lotdf = pd.concat([lotdf,df_lotfileInfo])
            
    lotEnd = {
        "LotID": [lot],
        "CarrierID": [""],
        "TimeStamp": [end_date_time],
        "Message": ["Lot End"],
        "WaferID": [""],
        "SlotNumber": [""],
        "Date": [end_date_time.date()],
        "Time": [end_date_time.time()]
    }
    df_lotend = pd.DataFrame.from_dict(lotEnd)
    lotdf = pd.concat([lotdf,df_lotend])

lotdf

,LotID,CarrierID,TimeStamp,Message,WaferID,SlotNumber,Date,Time
0,AE211776-OPER7220,,2022-05-22 11:05:03,Lot Start,,,2022-05-22,11:05:03
0,AE211776-OPER7220,007762310000IMEC,2022-05-22 11:17:50,,@2,2,2022-05-22,11:17:50
0,AE211776-OPER7220,007762310000IMEC,2022-05-22 11:31:25,,@3,3,2022-05-22,11:31:25
0,AE211776-OPER7220,007762310000IMEC,2022-05-22 11:44:59,,@4,4,2022-05-22,11:44:59
0,AE211776-OPER7220,007762310000IMEC,2022-05-22 11:58:32,,@5,5,2022-05-22,11:58:32
0,AE211776-OPER7220,007762310000IMEC,2022-05-22 12:12:05,,@6,6,2022-05-22,12:12:05
0,AE211776-OPER7220,,2022-05-22 12:12:05,Lot End,,,2022-05-22,12:12:05
0,AL201894-B,,2022-05-23 12:15:00,Lot Start,,,2022-05-23,12:15:00
0,AL201894-B,005463290000IMEC,2022-05-23 12:34:29,,@14,14,2022-05-23,12:34:29
0,AL201894-B,,2022-05-23 12:34:29,Lot End,,,2022-05-23,12:34:29


In [22]:
lotdf['SlotNumber'] = pd.to_numeric(lotdf['SlotNumber'])

In [6]:
output = lotdf.to_csv("C:/Users/Francis John Picaso/Repositories/Teo/AFM/AFM.csv",index=False,encoding = "utf-8")
print(output)

None


In [27]:
file = "C:/Users/Francis John Picaso/Repositories/Teo/toTest/220523_045932_E222056-7226_Slot06_/Info/Slot06_Info.csv"
columnPlaceholder = ['col' + str(i) for i in range(10)]
df = pd.read_csv(file,names=columnPlaceholder)
df

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9
0,Port No,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Carrier ID,005350860000IMEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Slot No,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lot ID,E222056-7226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sample ID,Slot06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Recipe ID,PLINE200\PATTERN\A-B-C\CUOXBOND\ICPAD_5_270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Tip Port No,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Tip Cassette ID,Cassette1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Tip Slot No,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Site ID,Site X,Site Y,Point No,X (um),Y (um),Method ID,State,FileName,Date


In [23]:
carrier_id = df[df["col0"] == "Carrier ID"]["col1"].values[0]
carrier_id

'007762310000IMEC'

In [42]:
#look for Tip Slot No
index = df[df["col0"] == "Tip Slot No"].index[0]
steps_header = index + 1
new_header = df.iloc[steps_header] #grab the first row for the header
steps = df[steps_header + 1:] #take the data less the header row
steps.columns = new_header #set the header row as the df header
steps.iloc[-1:]["Date"].values[0]

'2022/05/23 05:16:34'

In [12]:
#Azure blog
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__


accountName = "fpallpurpose"
accountKey = "XQKpky5uRpP3sKvnXskS8ZvhxJZnncWPsrpIeOxi44Ni4eZXPbAuUyc8J0jWwgVappzn4XBAaFko+AStfVw0eg=="
containerName = "teo"
blobName = "test3.json"
fileName = "AFM.csv"
path = "C:/Users/Francis John Picaso/Repositories/Teo/AFM/AFM.csv"


In [13]:
connection_string='DefaultEndpointsProtocol=https;AccountName=fpallpurpose;AccountKey=XQKpky5uRpP3sKvnXskS8ZvhxJZnncWPsrpIeOxi44Ni4eZXPbAuUyc8J0jWwgVappzn4XBAaFko+AStfVw0eg==;EndpointSuffix=core.windows.net'
# Instantiate a new BlobServiceClient using a connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)


blob_client = blob_service_client.get_blob_client(container=containerName, blob=fileName)

print("\nUploading to Azure Storage as blob:\n\t" + fileName)

# Upload the created file
with open(path, "rb") as data:
    blob_client.upload_blob(data)


Uploading to Azure Storage as blob:
	AFM.csv


In [14]:
from azure.storage.filedatalake import DataLakeServiceClient

In [29]:
lotdf['SlotNumber'] = pd.to_numeric(lotdf['SlotNumber'])
lotdf['Date'] = pd.to_datetime(lotdf['Date'], format='%Y-%m-%d')
lotdf['Time'] = pd.to_datetime(lotdf['Time'], format='%H:%M:%S')

lotdf.dtypes

LotID                 object
CarrierID             object
TimeStamp     datetime64[ns]
Message               object
WaferID               object
SlotNumber           float64
Date          datetime64[ns]
Time          datetime64[ns]
dtype: object

In [30]:
#needs fastparquet 
lotdf.to_parquet('C:/Users/Francis John Picaso/Repositories/Teo/AFM/afm.parquet.gzip',compression='gzip')  

In [15]:
connectionstring = "DefaultEndpointsProtocol=https;AccountName=fjpdatalakestore;AccountKey=rNUm/qKWf/7/YjuVWwM9w4qqMt+EMcJWcy0+XkNxbCDwNtyHsjD7jRIxIwm1uoEoSdMWfS+z9cpL+AStZ+/GYA==;EndpointSuffix=core.windows.net"

service = DataLakeServiceClient.from_connection_string(conn_str=connectionstring)

In [35]:
file_system_client = service.get_file_system_client(file_system="teo")

directory_client = file_system_client.get_directory_client("logs")
        
file_client = directory_client.get_file_client("AFM.csv")

local_file = open("C:/Users/Francis John Picaso/Repositories/Teo/AFM/AFM.csv",'r')

file_contents = local_file.read()

file_client.upload_data(file_contents, overwrite=True)

{'date': datetime.datetime(2022, 5, 29, 4, 46, 36, tzinfo=datetime.timezone.utc),
 'etag': '"0x8DA412E3700154F"',
 'last_modified': datetime.datetime(2022, 5, 29, 4, 46, 37, tzinfo=datetime.timezone.utc),
 'content_length': 0,
 'client_request_id': '5284533a-df0a-11ec-b70e-a8934a98a180',
 'request_id': '5c7002f9-a01f-000d-2a17-73282a000000',
 'version': '2021-06-08',
 'request_server_encrypted': False,
 'encryption_key_sha256': None}

In [50]:
from io import BytesIO

parquet_file = BytesIO()
lotdf.to_parquet(parquet_file, engine = 'pyarrow')
parquet_file.seek(0)

container = 'teo'
blob_path = 'afm.parquet'

connectionstring = "DefaultEndpointsProtocol=https;AccountName=fjpdatalakestore;AccountKey=rNUm/qKWf/7/YjuVWwM9w4qqMt+EMcJWcy0+XkNxbCDwNtyHsjD7jRIxIwm1uoEoSdMWfS+z9cpL+AStZ+/GYA==;EndpointSuffix=core.windows.net"

# Instantiate a new BlobServiceClient using a connection string
blob_service_client = BlobServiceClient.from_connection_string(connectionstring)


blob_client = blob_service_client.get_blob_client(container = container, blob = blob_path)
blob_client.upload_blob(data = parquet_file)

{'etag': '"0x8DA4130DFD197CB"',
 'last_modified': datetime.datetime(2022, 5, 29, 5, 5, 39, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'-<3\x1e\x0e#\xf8Y\xa0{\x1d\xf6\x10\t\xb1\xf8'),
 'client_request_id': 'fa97964a-df0c-11ec-b949-a8934a98a180',
 'request_id': 'c6cff4d9-801e-0025-2419-734982000000',
 'version': '2021-06-08',
 'version_id': None,
 'date': datetime.datetime(2022, 5, 29, 5, 5, 39, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}